# Module 

In [1]:
import pandas as pd 
import numpy as np 
from faker import Faker
import random
from datetime import datetime
import time
from sklearn.utils import shuffle
import shortuuid

# Create-Function

In [2]:
fake = Faker('DE')


def create_df(df_len, churn_per=0.1, dupl=True, nans=False, save=True):

    start = time.time()
    assert int(df_len)
    churn_freq = int(round(df_len * churn_per))
    not_churn_freq = int(round(df_len - churn_freq))

    #################################

    def create_data(x, y):

        city = []
        name = []
        seit = []
        kd_nummer = []
        kd_gruppe = []

        kd = ['Kleinkunde', 'mittlerer Kunde', 'Großkunde']

        churn = ['1']
        churn_list = []
        not_churn = ['0']
        not_churn_list = []

        len_df = int(x)

        if y == 'kdnum':
            kd_nummer = random.sample(range(1000, 100000), len_df)
            #kd_nummer.append(b)

            #for _ in range(x):
            #b=random.sample(x)
            #kd_nummer.append(b)

        if y == 'city':
            for _ in range(x):
                b = fake.city()
                city.append(b)
        if y == 'date':
            for _ in range(x):
                b = fake.date()
                seit.append(b)
        if y == 'name':
            for _ in range(x):
                b = fake.name()
                name.append(b)
        if y == 'kdgrp':
            for _ in range(x):
                b = random.choice(kd)
                kd_gruppe.append(b)

        if y == 'churn':
            for _ in range(churn_freq):
                b = random.choice(churn)
                churn_list.append(b)
            for _ in range(not_churn_freq):
                a = random.choice(not_churn)
                not_churn_list.append(a)
            c_list = churn_list + not_churn_list
            random.shuffle(c_list)


        return kd_nummer if len(kd_nummer)!=0 else name if len(name)!=0 else \
                    seit if len(seit)!=0 else city if len(city)!=0 else kd_gruppe if len(kd_gruppe)!=0\
                    else c_list

    ###########################################################

    def modify_data(df, kd_grp='Kleinkunde', alter=True, city=0.4):

        kd = ['Kleinkunde', 'mittlerer Kunde', 'Großkunde']

        kd_diff = [x for x in kd if x not in kd_grp]

        for a, b in enumerate(kd_diff):
            df.loc[(df['Kündigung']=='1') &(df['Kundengruppe']==b).sample(frac=.4),\
                'Kundengruppe']=kd_grp
            if a == 1:
                df.loc[(df['Kündigung']=='1') &(df['Kundengruppe']==b).sample(frac=.2),\
                'Kundengruppe']=kd_grp

        df['Anzahl_Mieteinheiten']=np.where(df['Kundengruppe']=='Kleinkunde',\
                                            np.random.randint(2,50,len(df)),
                                            (np.where(df['Kundengruppe']=='mittlerer Kunde',
                                            np.random.randint(51,200,len(df)),
                                            (np.where(df['Kundengruppe']=='Großkunde',
                                                np.random.randint(201,1000,len(df)),'leer')
                                            ))))

        ###
        df['Jahr'] = pd.DatetimeIndex(df['Im Bestand seit']).year

        ###
        df['Vertrag'] = datetime.now().year - df['Jahr']

        ###
        df['Vertragsperiode'] = round((round(df['Vertrag']) / 10) - 0.5)

        ###
        df['Vertragsperiode'] = df['Vertragsperiode'].astype(int)

        ##
        df['Kundentyp'] = np.where(
            df['Vertragsperiode'] == 0, 'Neukunde', (np.where(
                (df['Vertragsperiode'] >= 1) &
                (df['Vertragsperiode'] <= 3), 'Bestandskunden', 'Altkunden')))
        ####
        if alter:
            df.loc[(df['Kündigung']=='1') &(df['Kundentyp']=='Altkunden').sample(frac=.8),\
                'Kundentyp']='Neukunde'

        ###
        city_list = []

        unique_city = df['Stadt'].unique()

        for _ in range(int(len(unique_city) * city)):
            b = random.choice(unique_city)
            city_list.append(b)

        diff=pd.DataFrame(set(df['Stadt'].to_list())-set(city_list))\
                                    .sample(frac=city)[0].to_list()

        df['Konkurrenz'] = np.where(
            df['Stadt'].isin(city_list), 'Hohe Konkurrenz',
            (np.where(df['Stadt'].isin(diff), 'mittlerer Konkurrenz',
                      'Keine Konkurrenz')))

        df.loc[(df['Konkurrenz'] == 'Hohe Konkurrenz') &
               (df['Kündigung'] == '0').sample(frac=city), 'Kündigung'] = '1'

        return df.drop(['Jahr', 'Vertragsperiode','Vertrag','Im Bestand seit'], axis=1)

    ##################################################################

    def complaints(df, per=0.3, per_heavy=0.8):

        #
        interakt_list = []
        bs_list = []
        year_list = []
        df_interakt = df[['Kundennummer', 'Kündigung']]
        bs = [
            'leichte Beschwerde', 'mittlere Beschwerde', 'schwere Beschwerde'
        ]

        complaint_range = int(round(len(df_interakt) * per))

        for _ in range(complaint_range):
            df_sample = df_interakt.sample(frac=per, random_state=42)
            interakt_list.append(df_sample)

        df_interaktion = pd.DataFrame(np.concatenate(interakt_list))

        for i in range(len(df_interaktion)):
            g = random.choice(bs)
            bs_list.append(g)
            # Rechenintensiv!
            h = shortuuid.uuid()+'{}'.format(i)
            year_list.append(h)

        df_bs_list = pd.DataFrame(bs_list).rename(columns={0: 'Beschwerden'})

        year_df = pd.DataFrame(year_list).rename(columns={0: 'Beschwerde_id'})

        df_interaktion = df_interaktion.merge(df_bs_list,
                                              left_index=True,
                                              right_index=True)
        df_interaktion = df_interaktion.merge(year_df,
                                              left_index=True,
                                              right_index=True)

        df_interaktion.loc[(df_interaktion[1]=='0')&(df_interaktion['Beschwerden']=='schwere Beschwerde')\
                   .sample(frac=1.0),'Beschwerden']='leichte Beschwerde'

        df_interaktion.loc[(df_interaktion[1]=='1')&(df_interaktion['Beschwerden']!='schwere Beschwerden')\
                   .sample(frac=1.0),'Beschwerden']='schwere Beschwerde'

        return df_interaktion.rename(columns={0: 'KD.NR'}).drop([1], axis=1)

    #####################################################

    def geräte(df):
        df_geräte = df[['Kundennummer', 'Kündigung', 'Kundentyp']]

        Geräte = ['DeviceGen2', 'DeviceGen3', 'DeviceGen4']

        ger_list = []
        for _ in range(len(df)):
            a = random.choice(Geräte)
            ger_list.append(a)

        df_device = pd.DataFrame(ger_list)

        df_geräte = df_geräte.merge(df_device,
                                    left_index=True,
                                    right_index=True)

        df_geräte.loc[(df_geräte['Kündigung']=='1')&(df_geräte['Kundentyp']=='Neukunde')\
                       .sample(frac=1.0),0]='DeviceGen4'
        
        df_geräte.loc[(df_geräte['Kündigung']=='0')&(df_geräte['Kundentyp']=='Neukunde')\
                       .sample(frac=1.0),0]='DeviceGen3'

        df_geräte.loc[(df_geräte['Kundentyp']=='Neukunde')&(df_geräte[0]=='DeviceGen2')\
                       .sample(frac=1.0),0]='DeviceGen3'

        df_geräte.loc[(df_geräte['Kundentyp']=='Altkunden')&(df_geräte[0]=='DeviceGen4')\
                       .sample(frac=1.0),0]='DeviceGen3'

        df_geräte.loc[(df_geräte['Kundentyp']=='Altkunden')&(df_geräte[0]=='DeviceGen3')\
                       .sample(frac=1.0),0]='DeviceGen2'

        df_geräte.loc[(df_geräte['Kundentyp']=='Bestandskunden')&(df_geräte[0]=='DeviceGen4')\
                       .sample(frac=1.0),0]='DeviceGen3'

        return df_geräte.drop(['Kündigung', 'Kundentyp'],
                              axis=1).rename(columns={0: 'Gerätetyp'})

    ####################################################################################

    df_len = int(df_len)

    df = pd.DataFrame(create_data(
        x=df_len, y='kdnum')).rename(columns={0: 'Kundennummer'})
    df_name = pd.DataFrame(create_data(
        x=df_len, y='name')).rename(columns={0: 'Kundenname'})
    df_stadt = pd.DataFrame(create_data(x=df_len,
                                        y='city')).rename(columns={0: 'Stadt'})
    df_date = pd.DataFrame(create_data(
        x=df_len, y='date')).rename(columns={0: 'Im Bestand seit'})
    df_grp = pd.DataFrame(create_data(
        x=df_len, y='kdgrp')).rename(columns={0: 'Kundengruppe'})
    df_churn = pd.DataFrame(create_data(
        x=df_len, y='churn')).rename(columns={0: 'Kündigung'})

    for frame in df_name, df_stadt, df_date, df_grp, df_churn:
        df = df.merge(frame, left_index=True, right_index=True)

    print('Data created')

    #####################################

    df = modify_data(df)

    print('Data modified')

    df_complaints = complaints(df)

    print('Complaints created')

    df_devices = geräte(df)

    print("Die Erstellungszeit für {} Fälle liegt bei {} Minuten.".format(
        len(df), round(round(time.time() - start) / 60)))

    #####################################

    if dupl:

        ## Duplicated

        # Erstelle 20 Duplikate
        df_dupl = df[:20]
        df = pd.concat([df, df_dupl])

    if nans:

        ## NaN's

        # Col-Missings
        nan_list = ['Anzahl_Mieteinheiten']

        nan_col = [x for x in df.columns if x in nan_list]

        for col in nan_col:
            df.loc[df.sample(frac=0.1).index, col] = np.nan

    # Shuffle set
    df = shuffle(df).reset_index(drop=True)

    # List for saving
    df_name = ['Basetable', 'Complaints', 'Devices']

    # Savingloop
    if save:
        for i, j in zip(df_name, [df, df_complaints, df_devices]):
            j.to_csv('{}.csv'.format(i), sep=';', encoding='latin-1')

    return df, df_complaints, df_devices

In [3]:
df,beschwerde, geräte=create_df(10000)

Data created
Data modified
Complaints created
Die Erstellungszeit für 10000 Fälle liegt bei 2 Minuten.
